In [1]:
from bs4 import BeautifulSoup, NavigableString
import csv
import pandas as pd

In [3]:
# Record titles
html_article = '/Users/richardpang/Desktop/extract_maintext/extract_maintext/JM dataset/JM2021-2020.html'
titles_abstract = []
with open(html_article, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
titles=soup.find_all('h1',{'class':"delivery-title"})


for i in titles:
    title = i.get_text().replace(u'\xa0', u'')
    # Get abstract
    abstract = i.findNext('p').get_text()
    titles_abstract.append([title,abstract])

    
# Read articles
records = soup.find(id = "records")

# Delete keywords
for p in records.find_all('p', {"class":"body-paragraph"}):
    if p.get_text()[:9] == 'Keywords:':
        p.string = ''
    if p.get_text()[:18] == 'Online Supplement:':
        p.string = ''

# Delete tables
for table in records.find_all('table'):
    i = table.extract()
txt = records.get_text()



In [58]:
len('Online Supplement:')

18

In [59]:
len(titles_abstract)

57

# Citation scraping

In [60]:
import csv
import re
import time
from time import sleep
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [61]:
def make_url(keyword, conf, author, year, paper_id=None):
    """make url for search papers
    normal search (keyword, conf, author, year) or target search (paper_id)
    :param keyword: str or None
    :param conf: str or None, conference information
    :param author: str or None, author information
    :param year: int or None, published year
    :param paper_id: None or int, paper information
    :return: url
    """
    assert (
        keyword is not None
        or conf is not None
        or author is not None
        or year is not None
        or paper_id is not None
    ), "KeywordNotFoundError"
    url = "https://scholar.google.co.jp/scholar?"
    if paper_id is not None:
        url += f"&cites={paper_id}"
    else:
        url += "&as_sdt=0%2C5"
        if keyword is not None:
            url += f"&as_q={'%20'.join(keyword.split())}"
        else:
            url += "&as_q="
        if conf is not None:
            url += f"&as_publication={'%20'.join(conf.split())}"
        if author is not None:
            author = "+".join(author.split())
            url += f"&as_sauthors={'%20'.join(author.split())}"
        if year is not None:
            url += f"&as_ylo={year}"
    return url

In [62]:
def get_citations(soup):
    """obtain number of citations from soup
    :param soup: parsed html by BeautifulSoup
    :return: ci_num_list
    """
    tags3 = soup.find_all(text=re.compile("被引用"))
    ci_num_list = []
    for tag3 in tags3:
        # 被引用数取得
        citation = tag3.replace("被引用", "")[1:3]
        ci_num_list.append(int(citation))
    return ci_num_list

In [63]:
cite_data = []
for title in titles_lst:
    keyword = title
    url = make_url(keyword=keyword, conf=None, author=None, year=None)
    html_doc = requests.get(url).text
    soup = BeautifulSoup(html_doc, "html.parser")
    # print(get_citations(soup)
    cite_data.append([title,get_citations(soup)[0],len(get_citations(soup))]) 
    sleep(np.random.randint(5, 10))
df = pd.DataFrame(cite_data)
df.to_csv('/Users/richardpang/Desktop/extract_maintext/extract_maintext/citation_data.csv')


KeyboardInterrupt: 

# Data Processing

In [64]:
new_txt = txt.replace('Database:\n\t\t\t\t\t\tBusiness Source Complete', "partitionmark")
new_txt = new_txt.replace('Footnotes', "partitionmark")
new_txt = new_txt.replace('Supplemental Material', "partitionmark")
partition = new_txt.split('partitionmark')
# print("partition", len(partition))


In [65]:
len(partition[1])

70605

In [68]:
f = open('articles50.csv', 'w')
writer = csv.writer(f)
header = ['Number', "Title", 'Abstract','Body' ]
writer.writerow(header)
count = 1
for i in range (len(partition)):
    # print(i,len(partition[i]))
    text = partition[i]

    if len(text) > 20000:
        for pair in titles_abstract:
            title = pair[0]
            abstract = pair[1]
            if text[:50] in title:
                text = text[len(title)+1:]
                text = text[len(abstract):]


                row = [count,title,abstract,text]
                writer.writerow(row)
                count += 1
        # print(text)
        # if text[0].isupper():
        # print(count,partition[i][:30])
        # partition[i] is the main text
        # abstract = text.split('KEYWORDS_SPLIT',1)[0]
        # body = text.split('KEYWORDS_SPLIT',1)[1]
        # for title in titles_lst:
        #     if abstract[:50] in title:
        #         # print(title,abstract[:50])
        #         # print(abstract[:50])
        #         abstract = abstract[len(title):]
        #         # print(abstract)
        #         row = [count, title, abstract, body]
        #         writer.writerow(row)
        #         count += 1
print(count-1)

44


In [7]:
my_file = 'JM outpout/2016-2020JM.csv'
df = pd.read_csv(my_file)
df

,Number,Word Count,Title,Abstract,Body
0,1,69316,(When) Does Third-Party Recognition for Design...,Third-party recognition for design excellence ...,Product design is a critical aspect of the inn...
1,2,70088,A Cinderella Story: How Past Identity Salience...,"Like Cinderella, many repurposed products invo...",A worn-out airbag becomes a backpack. A leaky ...
2,3,20790,A Commentary on “Transformative Marketing: The...,Transformative: Causing a marked change in som...,As a complement to the transformative marketin...
3,4,64658,A Meta-Analysis of When and How Advertising Cr...,Although creativity is often considered a key ...,Creativity is important in marketing and is of...
4,5,82765,A Study of Bidding Behavior in Voluntary-Pay P...,The authors investigate compliance behavior an...,"In a voluntary-pay auction format, all bidders..."
...,...,...,...,...,...
136,137,68932,Who Is Wary of User Design? The Role of Power-...,This article evaluates when a user-design appr...,What do Threadless and Linux have in common? T...
137,138,73376,Who Receives Credit or Blame? The Effects of M...,While prior research has found that consumer-i...,Imagine that a consumer goes to a well-known s...
138,139,57510,Why the Dynamics of Competition Matter for Cat...,Category management (CM) has become a widespre...,Organized retail is an important business sect...
139,140,70489,Why Unhappy Customers Are Unlikely to Share Th...,"For brands to thrive, they must understand con...","For brands to succeed, it is imperative that t..."
